# Goals for the upcoming milestone: focus on Switzerland
* Load all relevant FAO and Impex data, and justify why we used these sources (as opposed to the Federal Statistics Office data, which gives related numbers but does not match up with what is provided by FAO and Impex)
* From the following article, copy the emissions intensities by food group world averages data into an excel file, and load that data (we will use the global average for all countries, since by country is not available for all types of food): https://www.sciencedirect.com/science/article/pii/S0959652616303584
* How much does Switzerland consume? -- calculate from imports, exports, and domestic production
* How much does Switzerland produce vs. supply its needs with imports?
* Update the README

# Introduction

There will be some text here to introduce the notebook, but I'm not sure what will go in the README vs here, so this is just a placeholder for now :)

**Assumptions of this data analysis:**
* Using global average of food production emissions for every single country, which is not at all realistic
* Only dealing with major food groups to simplify the analysis (for example, not looking at nuts, oils, etc--just fruits, vegetables, grains, meats, and non-meat animal products)
* etc

In [73]:
# import external libraries
%matplotlib inline
import collections
import inspect
import pickle
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
# import local dependencies
from scripts.helpers import *
from scripts.plots import *
from scripts.data_manipulation import *
from scripts.data_analysis import *

In [113]:
impex = load_impex()

                      quantity        value  quantity.1    value.1
commercial_partner                                                
Argentina             209002.0     746881.0         0.0        0.0
Austria               464034.0    3067489.0    314324.0  3649800.0
Belgium                11884.0      69658.0         0.0        0.0
Brazil              15785585.0   41794159.0         0.0        0.0
Czech Republic         20514.0      99641.0        12.0       43.0
Denmark               157200.0     809352.0         0.0        0.0
Emirates, Arab             0.0          0.0         6.0      132.0
France               3351364.0   19899073.0       550.0     4867.0
Germany              5268553.0   24960575.0   2799551.0  1170752.0
Greece                   733.0       2824.0         0.0        0.0
Hungary              3092673.0   16410517.0         0.0        0.0
Italy                 193975.0     840379.0        23.0      150.0
Kosovo                  3562.0       2494.0    320272.0   2580

***
**Exploratory data analysis**
***

The first step is to calculate, for each type of food, how much of what is consumed by the Swiss population is produced within Switzerland versus imported. To do this, we will combine 3 sets of data: imports, exports, and domestic production. The imports and exports data are sourced from [Swiss Impex](https://www.gate.ezv.admin.ch/swissimpex/index.xhtml), a website hosted by the Swiss Federal Customs Administration which provides data on Switzerland's global trade activity. Domestic production data comes from [FAOStat](http://www.fao.org/faostat/en/#data), the Food and Agriculture Organization of the United Nations which offers a variety of agricultural-related data. In theory, the amount of food consumed in Switzerland (including food waste) can be calculated from these datasets:

Food consumed = domestic production + imports - exports

Note that the Swiss Federal Statistics Office also provided relevant data--namely, it has a dataset on Swiss food consumption by type of food. Unfortunately, these data conflicted with the data from FAO and Swiss Impex. Since the Federal Statistics Office data was much less detailed (for instance, more general/broad food categories), we decided to focus on Impex and FAO, knowing that the numbers must be taken with a grain of salt since it is difficult to accurately quantify such data.

Let's load all the data and then combine the various data sets to get the values of interest.

In [4]:
# First, load the data on domestic production: what types and how much of each crop does Switzerland grow?
# This data comes from FAO
swiss_crop_prod = pd.read_csv("../data/swiss_crop_production_FAO.csv")
swiss_crop_prod

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QC,Crops,211,Switzerland,5510,Production,515,Apples,2017,2017,tonnes,225622,Im,FAO data based on imputation methodology
1,QC,Crops,211,Switzerland,5510,Production,526,Apricots,2017,2017,tonnes,9420,Im,FAO data based on imputation methodology
2,QC,Crops,211,Switzerland,5510,Production,366,Artichokes,2017,2017,tonnes,0,NaN,Official data
3,QC,Crops,211,Switzerland,5510,Production,367,Asparagus,2017,2017,tonnes,722,NaN,Official data
4,QC,Crops,211,Switzerland,5510,Production,44,Barley,2017,2017,tonnes,207374,NaN,Official data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,QC,Crops,211,Switzerland,5510,Production,97,Triticale,2017,2017,tonnes,51424,NaN,Official data
57,QC,Crops,211,Switzerland,5510,Production,463,"Vegetables, fresh nes",2017,2017,tonnes,54812,NaN,Official data
58,QC,Crops,211,Switzerland,5510,Production,420,"Vegetables, leguminous nes",2017,2017,tonnes,69,NaN,Official data
59,QC,Crops,211,Switzerland,5510,Production,222,"Walnuts, with shell",2017,2017,tonnes,2528,Im,FAO data based on imputation methodology


# TODO: download and load here domestic production of meats, animal products

# TODO: download and then load here all datasets for other food types (meat, animal products, grains/cereals)

Then, the idea going from here onwards: merge the various datasets so that we eventually have one dataframe with each type of food (e.g. bananas, potatoes, etc.) and one column for how much is grown domestically and not exported, and another column for how much is imported. Maybe also do some pretty visualizations, like make a ratio of those two columns and then plot to see if there are any identifiable clusters?

***

After finishing what is listed in the cell above, we can go further into animal feed.....

One interesting aspect of meat and non-meat animal product production, which is not relevant to the other food groups, is that animal feed may be sourced from a different location than where the meat/animal product is produced. This implicates the sourcing of animal feed as a very important factor for the carbon intensity of these foods. For example, if Switzerland produced most of its meat domestically, but it imported all of its feed, the carbon intensity of its meat would be much higher than if the feed were grown domestically. Thus, both aspects of the final food product must be examined.

Let's now examine Swiss production, imports, and exports of *animal feed*. 

In [ ]:
%psource load_imported_feed

In [ ]:
imported_feed = load_imported_feed()
imported_feed.head()

# TODO: download and load here the datasets for animal feed exported and grown domestically

In [ ]:
ratio_nearby_feed = compute_nearby_imports_ratio(imported_feed)
print("{0:.1%} of imported feed comes from nearby countries".format(ratio_nearby_feed))

Nearly three-quarters of imported animal feed is imported from nearby countries, which means when Switzerland must import feed, they mostly minimize carbon emissions from the process by reducing the travel distance of the feed. More importantly, a report from the State Secretariat for Economic Affairs ("Concentrate Animal Feed as an Input Good in Swiss Agricultural Production - The Effects of Border Protection and Other Support Measures") claims 90% of animal feed used in Switzerland is domestically produced, drastically reducing the carbon impact of the meat industry in the country. This percentage can be found in [this PDF document](https://www.sbv-usp.ch/fileadmin/sbvuspch/00_Bilder/06_Services/Agristat/Statistiken/Produktionsmittel__Umwelt/SES2018_Kap04_Produktionsmittel-Umwelt.pdf) (see the table on page 8):

We will need to pay attention to where food is coming from by animal type. For example, 90% of cow and cattle feed is domestically produced, whereas that number is only 52% for pork and 30% for poultry (found in the Concentrate Animal Feed report above, where it was sourced from the PDF linked above). So even if chickens are less carbon intensive, they require more imported feed which might increase their carbon output!

As we can see from the above table, Switzerland imported 77 million kg of meat and edible offal in 2018 (equivalent to 77 thousand tonnes).